In [74]:
import numpy as np
import pandas as pd
from os import getcwd
from os.path import join, dirname
import string_utils
import datetime
from bokeh.plotting import figure, show, output_notebook
%matplotlib inline

ONE_DAY = datetime.timedelta(days=1)
DATA_DIR = join(dirname(getcwd()), 'data')
DATA_FILE = join(DATA_DIR, 'WebsiteUploads_dbo_ProductInventory_010.csv')
headers = ['sku', 'bulk_id', 'size_code', 'quantity_on_hand', 'lifecycle_status_flag', 'expected_date',
           'location', 'back_order_type', 'product_code', 'product_type', 'is_perishable', 'is_dropship',
           'street_address_required', 'effective_on', 'expired_on', 'added_on', 'added_by', 'modified_on',
           'modified_by', 'version']
column_types = {
    'sku': str, 
    'bulk_id': str, 
    'size_code': str,
    'is_perishable': bool,
    'is_dropship': bool,
    'street_address_required': bool
}
df = pd.read_csv(DATA_FILE, names=headers, parse_dates=[5, 13, 14, 15, 17], dtype=column_types)


OUT_OF_STOCK_FLAGS = ['DS', 'ED']
BACKORDERED_FLAGS = ['DX', 'EX']
STOCK_STATUS_LABELS = {
    0: 'out_of_stock',
    1: 'in_stock',
    2: 'back_ordered'
}

# todo: review this logic
def assign_stock_status(row):
    """determine the stock status of a sku"""
    if row['lifecycle_status_flag'] in OUT_OF_STOCK_FLAGS:
        return 0
    elif row['expected_date'] and row['lifecycle_status_flag'] in BACKORDERED_FLAGS:
        return 2
    else:
        return 1

def date_to_midnight(dt):
    """convert a date to a datetime set at midnight"""
    return datetime.datetime.combine(dt, datetime.time())

def datetime_to_date(dt):
    """convert a datetime to a date"""
    return dt.date()

def calculate_daily_duration(row):
    if row['effective_on_date'] != row['expired_on_date']:
        return date_to_midnight(row['effective_on_date'] + ONE_DAY) - row['effective_on']
    else:
        return row['expired_on'] - row['effective_on']

### Dataset cleansing
A rough, early estimate shows that we can reduce the row count of the dataset but as much as 2/5 by filtering some uninteresting
rows to begin with.

#### Filter website-hidden SKUs
A number of SKUs are hidden from sale on the website, which makes them, effectively, out of stock.  I'm filtering
them out here to avoid cluttering up the data.

#### Filtering 'always out of stock' items
Some SKUs were sold on the site at sometime in the past, but are no longer saleable items, and will probably remain
that way.  In any case, items that have been listed as 'out of stock' for the duration of the data set are removed.

In [75]:
df = df.drop(['location', 'back_order_type', 'product_code', 'product_type', 'is_perishable', 'is_dropship',
           'street_address_required', 'added_on', 'added_by', 'modified_on', 'modified_by', 'version'], axis=1)
# apply stock status
df['stock_status'] = df.apply(assign_stock_status, axis=1)

# remove skus we don't care about
df = df[~df['size_code'].isin(['99', '98', '97'])]
# remove skus that are always out of stock
stock_status_only = df[['sku', 'stock_status']]
status_counts = stock_status_only.pivot_table(index='sku', 
                                              columns=['stock_status'], 
                                              values='stock_status', 
                                              aggfunc=lambda x: x.count()).fillna(0)
for col in range(3):
    if col not in status_counts:
        status_counts[col] = 0
always_out_of_stock = status_counts[(status_counts[1] == 0) & (status_counts[2] == 0)].index.values
df = df[~df['sku'].isin(always_out_of_stock)]


### Preparation
Determine to which days the rows are assigned, and calculate the duration of each 

In [76]:
# facilitate grouping by day
df['effective_on_date'] = df['effective_on'].apply(datetime_to_date)
df['expired_on_date'] = df['expired_on'].apply(datetime_to_date)

# rows that have effective_on_date != expired_on_date need to apply to both the
# end of the day they were effective on, and the beginning of the day they expire
# on. add new rows for the 'expired on day' pull forward some information
add_rows = df[df['effective_on_date'] != df['expired_on_date']]
add_rows = add_rows.assign(effective_on_date=add_rows['expired_on_date'])
add_rows = add_rows.assign(effective_on=add_rows['effective_on_date'].apply(date_to_midnight))
df = df.append(add_rows, ignore_index=True)

# calculate the status duration
df['status_duration'] = df.apply(calculate_daily_duration, axis=1)

In [ ]:
df